# Map Reduce and Shuffling

### Introduction

In the last lesson, we saw how with Pyspark, we can partition our dataset across the cores of our executor.  This allows us to process data in a dataset in parallel.  In this lesson, we'll take a closer look at how Spark performs these operations across both nodes and cores.

### Getting Set Up (For Colab Only)

Now, later on in this lesson, we'll be using the Spark UI -- which is a sort of Spark dashboard -- to learn about how Spark works under the hood.  If we're using google colab, we'll need to run the following to eventually connect to this.

* Begin by installing some pip packages and the java development kit.

In [ ]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet 
!apt install openjdk-8-jdk-headless &> /dev/null

* Then set the java environmental variable

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

* Then connect to a SparkSession, setting the spark ui port to `4050`.

In [ ]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050').setAppName("films").setMaster("local[2]")
sc = SparkContext.getOrCreate(conf=conf)

* Then we need to install ngrok which will allow us to place our local spark ui on the web.

In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip &> /dev/null
!unzip ngrok-stable-linux-amd64.zip &> /dev/null
get_ipython().system_raw('./ngrok http 4050 &')

* And finally we get a link our Spark UI

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Map Reduce in Pyspark

Now from here, we'll create our list of movies in Python.

In [3]:
movies = ['Shazam!', 'Minari', 'Captain Marvel', 
          'Pulp Fiction', 'Casablanca', 'Michael Clayton',
          'Sicario']

And from there turn it into an RDD.

In [4]:
rdd = sc.parallelize(movies, 4)

Now, we haven't said so before but when we pass our data into spark using the `parellize` method, we create a resiliant distributed dataset -- a RDD.

In [5]:
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

As we saw in previous lessons, an RDD is a dataset that is distributed across cores in our execeutor.  And as we know, if we say use a `filter` to look for the movie 'Michael Clayton' across the entire dataset, we perform this filter task four times across the cluster in parallel.

<img src="./map_red_cluster.jpg" width="100%">

What we see from the diagram above is that we send this query to our driver, and then the driver instructs the executors to query their partition of the data.

So each core finds the matching records in it's relevant partition and returns the matched records to the spark driver, which returns them to us.

In [44]:
rdd.filter(lambda movie: movie =='Michael Clayton').collect()

['Michael Clayton']

If we perform a `map` operation, as opposed to a `filter` essentially the same thing occurs.

In [45]:
rdd.map(lambda movie: movie.upper()).collect()

['SHAZAM!',
 'MINARI',
 'CAPTAIN MARVEL',
 'PULP FICTION',
 'CASABLANCA',
 'MICHAEL CLAYTON',
 'SICARIO']

For example, above, we are capitalizing our data across four parts of our dataset simultaneously, and then gathering the sending the results from each partition to our driver, where our driver then sends the results back to us.

> This process of having each partition performing the same operation, and then combining the results is called `map reduce`.  

### Shuffling

Now one thing we should be aware when when performing our distributed operations is shuffling.  Shuffling occurs when an operation requires us to send our data across partitions to successfully perform a query.  Because this transmission of data can require sending data across worker nodes, it is often a time intensive.

So now let's talk about some operations that can cause shuffling.  Oftentimes, shuffling occurs when we need to perform a group by.  

For example, let's say we have an RDD consisting of a list of movies, partitioned across cluster like so.

* Partition 1
    * Shazam!
    * Minari
* Partition 2
    * Captain Marvel
    * Pulp Fiction
* Partition 3
    * Casablanca
    * Michael Clayton
* Partition 4
    * Sicario

So above is our distributed dataset, and now let's say our query is to group the movies together by their first letter.  Because some of the records we want grouped together start off on different nodes, this grouping will require sending some movies from one worker node to another so that they can reside together.

> For example, above we can see that to group movies with the letter m together, we need to collect movies `Minari` and `Michael Clayton` across the first and third partitions, and then group them together. 

In [46]:
grouped_rdd = rdd.groupBy(lambda movie: movie[0]).map(lambda x : (x[0], list(x[1])))

In [47]:
grouped_rdd.collect()

[('M', ['Minari', 'Michael Clayton']),
 ('S', ['Shazam!', 'Sicario']),
 ('C', ['Captain Marvel', 'Casablanca']),
 ('P', ['Pulp Fiction'])]

### Looking under the hood

Now we can get see some of these operations in action by looking at the Spark UI.  So let's reference our spark context and click on the link to the Spark UI.

In [48]:
sc

<SparkContext master=local[2] appName=films>

And then if we click on the Spark UI link, we'll see something like the following. 

> <img src="./completed_jobs.png" width="60%">

So we can see that we have completed three jobs so far -- one for our `filter`, one for our `map`, and one for our `groupby`.  Now let's scroll down so we can begin to see more details about one of these jobs.

<img src="./jobs.png" width="100%">

The jobs are listed in reverse order - meaning that our most recent job is listed as job id 2, at the top of the list.  This was our group by job.  

We can click on the blue link to see more details about the job.

> <img src="./staged_group_by.png" width="60%">

So if we click the drop down for `Dag visualization`, we can see a little bit more about what occurred in the job.  Let's look at Stage 2.  Stage 2 says `parallelize` -- telling us it involved running a task in parallel.  Then if we look at the below table to `Stage Id 2`, we can see that this parallel task involved our group by.  

Ok, now let's click on the stage 2 box to see a little more details about this stage.

<img src="./group_by_task.png" width="40%">

So here we can see that the first task in this stage is to read in the RDD.  And then the next task is to perform the group by.  If we scroll further down, we can see that it's in this stage that the shuffling occurs.

<img src="./shuffling.png" width="100%">

The light green in the chart above is for the compute time -- searching through and querying the collection.  And the light yellow describes shuffling write time, so this is where we can see that the shuffling occur, when our dataset is reorganized by the first letter.  

> The shuffle time is relatively small here as all partitions are located on our local computer.

Remember that our job comes from the following code.  So the groupby seems to have occurred in Stage 2.

In [53]:
rdd.groupBy(lambda movie: movie[0]).collect()

[('M', <pyspark.resultiterable.ResultIterable at 0x10a4b0a30>),
 ('S', <pyspark.resultiterable.ResultIterable at 0x10a583550>),
 ('C', <pyspark.resultiterable.ResultIterable at 0x10a583130>),
 ('P', <pyspark.resultiterable.ResultIterable at 0x10a5834c0>)]

* map statement

And now that would leave the `map` for stage 3.  So let's take a look at that.

In [54]:
rdd.groupBy(lambda movie: movie[0]).map(lambda x : (x[0], list(x[1]))).collect()

[('M', ['Minari', 'Michael Clayton']),
 ('S', ['Shazam!', 'Sicario']),
 ('C', ['Captain Marvel', 'Casablanca']),
 ('P', ['Pulp Fiction'])]

So we can see these last steps represented if we click on stage 3.

<img src="./shuffled_rdd.png" width="40%">

Above we can see that we started with our shuffled rdd from the previous stage.

In [49]:
rdd.groupBy(lambda movie: movie[0]).collect()

[('M', <pyspark.resultiterable.ResultIterable at 0x10a583a90>),
 ('S', <pyspark.resultiterable.ResultIterable at 0x10a620730>),
 ('C', <pyspark.resultiterable.ResultIterable at 0x10a6207c0>),
 ('P', <pyspark.resultiterable.ResultIterable at 0x10a620850>)]

And then the next step was our map statement.  And finally the last step was the collect which resulted in Python code.

In [51]:
rdd.groupBy(lambda movie: movie[0]).map(lambda x : (x[0], list(x[1]))).collect()

[('M', ['Minari', 'Michael Clayton']),
 ('S', ['Shazam!', 'Sicario']),
 ('C', ['Captain Marvel', 'Casablanca']),
 ('P', ['Pulp Fiction'])]

* Understanding Map partitions

Notice that the task in the middle was called map partitions.  As we look at our Spark UI, we'll tend to see this a lot.  Map partitions is quite similar to map.  But map will execute our code one time for each record.  So if we have 2,000 records our related code will be called 2000 times.  With map partitions, the function is called only once per partition.  so if those 2,000 records are divided into 20 partitions, then the related function is only called 20 times.  This is more efficient call.  And it's for this reason that spark will translate our functions into map partitions under the hood.

### Summary

In this lesson, we learned about Spark performs map reduce, operations that result in shuffling, and how to see these steps using the Spark UI.  We saw that by partitioning our dataset, Spark operations like filter and map across all partitions simultaneously.  And we also saw that one thing to be careful of is shuffling, which occurs when Spark needs to repartition the data to perform the operation.  This often happens with a group by operation.  Shuffling can be expensive because it can require sending data across different nodes in the cluster.

We also learned about map partition which is similar to map.  The difference is that instead of performing an opeeration for each record, with map partition Spark performs an operation once per partition, which is faster.

### Resources

[Shuffling Documentation](https://spark.apache.org/docs/latest/rdd-programming-guide.html#shuffle-operations)

[5 hour pyspark tutorial](https://www.youtube.com/watch?v=GFC2gOL1p9k&t=2743s)

[Map vs Map Partition](https://sparkbyexamples.com/spark/spark-map-vs-mappartitions-transformation/)

[Group by key](https://backtobazics.com/big-data/spark/apache-spark-groupbykey-example/)

[Pyspark Google Colab](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)